In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers



(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()


classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,3,3,input_shape=(32,32,3),activation='relu'))
classifier.add(BatchNormalization())##BatchNormalization
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32,3,3,activation='relu'))
classifier.add(BatchNormalization())##BatchNormalization
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100,activation='relu',kernel_regularizer=regularizers.l2(l=0.001))) ##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization
classifier.add(Dropout(p=0.5)) ##Dropout

classifier.add(Dense(output_dim=100,activation='relu',kernel_regularizer=regularizers.l2(0.001)))##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization

classifier.add(Dropout(p=0.3))##Dropout

classifier.add(Dense(output_dim=10,activation='softmax'))
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


from keras.preprocessing.image import ImageDataGenerator ##Augmentation
img_gen = ImageDataGenerator( featurewise_center=True,featurewise_std_normalization=True,rotation_range=10,width_shift_range=0.1,
                                            height_shift_range=0.1,shear_range=0.1,zoom_range=0.1,horizontal_flip=True,vertical_flip=False,dtype=np.float32)
img_gen.fit(x_train)


from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='test_loss', patience=8, verbose=1) ##earlystop

##開始訓練
classifier.fit_generator(img_gen.flow(x_train, y_train, batch_size=100) ,steps_per_epoch=500,
                               epochs=100, validation_data = (x_test, y_test),callbacks = [earlystop])

Using TensorFlow backend.
C:\Users\a0923\Anaconda3\envs\tensorflow114python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\a0923\Anaconda3\envs\tensorflow114python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\a0923\Anaconda3\envs\tensorflow114python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

(50000, 32, 32, 3)







C:\Users\a0923\Anaconda3\envs\tensorflow114python36\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., activation="relu")`


C:\Users\a0923\Anaconda3\envs\tensorflow114python36\lib\site-packages\ipykernel_launcher.py:46: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
C:\Users\a0923\Anaconda3\envs\tensorflow114python36\lib\site-packages\ipykernel_launcher.py:54: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
C:\Users\a0923\Anaconda3\envs\tensorflow114python36\lib\site-packages\ipykernel_launcher.py:56: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.5)`


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



C:\Users\a0923\Anaconda3\envs\tensorflow114python36\lib\site-packages\ipykernel_launcher.py:58: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
C:\Users\a0923\Anaconda3\envs\tensorflow114python36\lib\site-packages\ipykernel_launcher.py:61: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.3)`
C:\Users\a0923\Anaconda3\envs\tensorflow114python36\lib\site-packages\ipykernel_launcher.py:63: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
500/500 [==============================] - 23s 47ms/step - loss: 2.1090 - acc: 0.3521 - val_loss: 1.6198 - val_acc: 0.5008
Epoch 2/100
 13/500 [..............................] - ETA: 6s - loss: 1.8291 - acc: 0.4208

C:\Users\a0923\Anaconda3\envs\tensorflow114python36\lib\site-packages\keras\callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `test_loss` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


500/500 [==============================] - 20s 39ms/step - loss: 1.6628 - acc: 0.4827 - val_loss: 1.4607 - val_acc: 0.5463
Epoch 3/100
500/500 [==============================] - 20s 41ms/step - loss: 1.4814 - acc: 0.5354 - val_loss: 1.3658 - val_acc: 0.5713
Epoch 4/100
500/500 [==============================] - 20s 40ms/step - loss: 1.3727 - acc: 0.5679 - val_loss: 1.1994 - val_acc: 0.6237
Epoch 5/100
500/500 [==============================] - 20s 40ms/step - loss: 1.3004 - acc: 0.5903 - val_loss: 1.2112 - val_acc: 0.6183
Epoch 6/100
500/500 [==============================] - 20s 41ms/step - loss: 1.2568 - acc: 0.6031 - val_loss: 1.1802 - val_acc: 0.6257
Epoch 7/100
500/500 [==============================] - 20s 40ms/step - loss: 1.2248 - acc: 0.6108 - val_loss: 1.1159 - val_acc: 0.6512
Epoch 8/100
500/500 [==============================] - 20s 39ms/step - loss: 1.2041 - acc: 0.6198 - val_loss: 1.0635 - val_acc: 0.6697
Epoch 9/100
500/500 [==============================] - 20s 41ms/ste

## 上方有用到一些避免Overfitting 的方法，有興趣的學員們可以參考這篇Medium:https://medium.com/@CinnamonAITaiwan/cnn%E5%85%A5%E9%96%80-overfitting-d10acd15ec21

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[0.00389634, 0.03561514, 0.27043524, 0.294611  , 0.03410938,
        0.24646202, 0.07232887, 0.00360016, 0.00442342, 0.03451842]],
      dtype=float32)